In [36]:
import pandas as pd
import numpy as np
import os

In [37]:
computer_path = os.getcwd()
path_input_data = os.getcwd()+os.path.sep+"Data"
path_EPCs = computer_path + os.path.sep + r"Data" +os.path.sep +r"EPCs"
saving_path = computer_path

## Input parameters

In [38]:
# specify the name of the local authorit(y/ies) you would like to calculate heat demand data for
# if no name specified, this is done for all local authorities in England and Wales.

local_authority = ["Cardiff"]
potential = True #potential variable set to True to calculate heat demand after considering energy efficiency measures

if len(local_authority)>0:
    output_file_name = "LSOAs_in_selected_LA"
else:
    output_file_name ="LSOAs_in_England_Wales"
    
if potential:
    output_file_name = output_file_name + "_before_EE_heat_demand"
else:
    output_file_name = output_file_name + "_after_EE_heat_demand"


### Import lookup table linking posctode, LSOA, MSOA and Region

In [39]:
OA_lookup_file = r"PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_feb_2018.csv"
OA_lookup_df = pd.read_csv(path_input_data+os.path.sep+OA_lookup_file, low_memory=False)
OA_lookup_df.drop(["pcd8", "pcds", "dointr", "doterm", "usertype", "lsoa11nm", "msoa11nm","ladnmw", "FID"], axis=1, inplace=True) 
OA_lookup_df.drop_duplicates(inplace=True)

region_lookup_file = r"laregionlookup376las.xls"
region_lookup_df = pd.read_excel(path_input_data+os.path.sep+region_lookup_file, sheet_name=1, header=6, usecols=[0, 1, 3])

OA_lookup_df = pd.merge(OA_lookup_df, region_lookup_df, left_on='ladcd', right_on='la_code', how='left')

OA_lookup_df = pd.merge(OA_lookup_df, region_lookup_df, left_on='ladnm', right_on='la_name', how='left')
OA_lookup_df["la_code_x"].fillna(OA_lookup_df["la_code_y"], inplace=True)
OA_lookup_df["la_name_x"].fillna(OA_lookup_df["la_name_y"], inplace=True)
OA_lookup_df["region_name_x"].fillna(OA_lookup_df["region_name_y"], inplace=True)
OA_lookup_df.drop(["ladcd", "ladnm", "la_code_y","la_name_y", "region_name_y"], axis=1, inplace=True)
OA_lookup_df.dropna(subset=["la_name_x"], inplace=True)
OA_lookup_df["pcd7"]=OA_lookup_df["pcd7"].str.replace(' ','')
OA_lookup_df.columns = ['PCD7', 'OA11CD', 'LSOA11CD', 'MSOA11CD', 'LAD11CD', 'Local Authority', 'Region']

if len(local_authority)>0:
    OA_lookup_df=OA_lookup_df.loc[OA_lookup_df['Local Authority'].isin(local_authority),:]

final_df = OA_lookup_df.loc[:, ["OA11CD", "LSOA11CD", "Local Authority"]].drop_duplicates()
final_df.head()

,OA11CD,LSOA11CD,Local Authority
384000,W00008807,W01001942,Cardiff
384001,W00010127,W01001943,Cardiff
384003,W00010184,W01001944,Cardiff
384004,W00009100,W01001759,Cardiff
384006,W00010210,W01001940,Cardiff


### Add rurality information

In [40]:
rurality_file = "census_OA_data_England_and_Wales.xlsx"
rurality_df = pd.read_excel(path_input_data+os.path.sep+rurality_file)
rurality_df = rurality_df[["N_Households", "Rurality", 'OA_Code']]
rurality_df["Rurality"].replace({'Village': 'Village, Town and Fringe',
                              'Town and Fringe': 'Village, Town and Fringe'}, inplace=True)
rurality_columns= ["Rurality_"+str(x) for x in rurality_df["Rurality"].unique() if x is not np.nan]
rurality_df = pd.get_dummies(rurality_df, columns=["Rurality"])

rurality_df.head()

,N_Households,OA_Code,Rurality_Hamlet & Isolated Dwellings,Rurality_Urban >10K,"Rurality_Village, Town and Fringe"
0,119,E00078837,0,0,1
1,122,E00078838,0,0,1
2,97,E00078839,0,0,1
3,78,E00078840,1,0,0
4,117,E00078841,0,0,1


In [41]:
final_df = pd.merge(rurality_df,final_df, left_on='OA_Code', right_on='OA11CD', how='right')
final_df = final_df.groupby('LSOA11CD').sum()
final_df["Rurality"] = final_df[rurality_columns].idxmax(axis=1)
final_df.drop(rurality_columns, axis=1, inplace=True)
LSOA_lookup_df = OA_lookup_df.loc[:,["LSOA11CD", "MSOA11CD", "LAD11CD", "Local Authority"]].drop_duplicates()
final_df = pd.merge(final_df,LSOA_lookup_df, left_index=True, right_on='LSOA11CD', how='left')
final_df.set_index("LSOA11CD", inplace=True, drop=True)
final_df.head()

,N_Households,Rurality,MSOA11CD,LAD11CD,Local Authority
LSOA11CD,,,,,
W01001694,878,Rurality_Urban >10K,W02000402,W06000015,Cardiff
W01001695,845,Rurality_Urban >10K,W02000402,W06000015,Cardiff
W01001696,683,Rurality_Urban >10K,W02000402,W06000015,Cardiff
W01001697,762,Rurality_Urban >10K,W02000402,W06000015,Cardiff
W01001698,896,Rurality_Urban >10K,W02000402,W06000015,Cardiff


### Add census data dwelling type by heating type

In [42]:
dwelling_heating_file = "CT0213.xls"

frame = []

dwelling_type = ["Detached", "Semi-detached", "Terraced", "Flat"]
heating_type = ["No central heating", "Gas boiler", 'Resistance heating', "Oil boiler", "Solid fuel boiler"]
count = 2

for dwelling in dwelling_type:
    df = pd.read_excel(path_input_data+os.path.sep+dwelling_heating_file, sheet_name=count, header=11, index_col=0)
    df.drop(df.columns[0], axis=1, inplace=True)
    df.drop(df.columns[5], axis=1, inplace=True)
    df.columns = [dwelling + " " +heating+ " 2011" for heating in heating_type] 
    df["Code"] = df.index.str.strip().str.split(' ').str.get(0)
    df.set_index("Code", inplace=True, drop=True)
    df.dropna(inplace=True)
    print(df.shape)
    count = count + 1
    frame.append(df)
dwelling_heating_df = pd.concat(frame, axis=1, sort=False)

(42314, 5)
(42314, 5)
(42314, 5)
(42314, 5)


In [43]:
final_df = pd.merge(final_df, dwelling_heating_df,left_index=True, right_index=True, how='left')
final_df.head()

,N_Households,Rurality,MSOA11CD,LAD11CD,Local Authority,Detached No central heating 2011,Detached Gas boiler 2011,Detached Resistance heating 2011,Detached Oil boiler 2011,Detached Solid fuel boiler 2011,...,Terraced No central heating 2011,Terraced Gas boiler 2011,Terraced Resistance heating 2011,Terraced Oil boiler 2011,Terraced Solid fuel boiler 2011,Flat No central heating 2011,Flat Gas boiler 2011,Flat Resistance heating 2011,Flat Oil boiler 2011,Flat Solid fuel boiler 2011
LSOA11CD,,,,,,,,,,,,,,,,,,,,,
W01001694,878,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,20.0,4.0,0.0,0.0,...,10.0,396.0,10.0,0.0,1.0,33.0,220.0,72.0,0.0,0.0
W01001695,845,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,6.0,0.0,0.0,0.0,...,2.0,152.0,2.0,0.0,0.0,17.0,290.0,286.0,8.0,0.0
W01001696,683,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,20.0,2.0,0.0,0.0,...,9.0,338.0,12.0,0.0,0.0,3.0,174.0,25.0,0.0,0.0
W01001697,762,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,13.0,0.0,0.0,0.0,...,17.0,466.0,5.0,0.0,0.0,7.0,124.0,51.0,0.0,0.0
W01001698,896,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,13.0,0.0,0.0,0.0,...,15.0,225.0,5.0,0.0,1.0,33.0,384.0,152.0,1.0,0.0


### Add area to LSOA

In [44]:
file = 'UK_2011_LSOA_with_area.csv'

In [45]:
area_df = pd.read_csv(path_input_data+os.path.sep+file, index_col=1, dtype={"Area":float})
area_df["Area"] = area_df["Area"]/1000000
area_df.rename(columns = {'Area':'Area (km2)'}, inplace=True)
area_df.head()

,OBJECTID,AREA_NAME,MaxSimpTol,MinSimpTol,OBJECTID_1,GeographyCode,KS601EW0001,KS601EW0002,KS601EW0003,KS601EW0004,...,KS601EW0011,KS601EW0012,KS601EW0013,KS601EW0014,KS601EW0015,Full_time_PC,Part_time_PC,Unemployed_PC,Retired_PC,Area (km2)
AREA_ID,,,,,,,,,,,,,,,,,,,,,
E01025921,1001,North West Leicestershire 006D,75,75.0,32810.0,E01025921,1151.0,170.0,473.0,94.0,...,13.0,3.0,5.0,0.0,10.0,41.094700,14.769765,1.650738,21.894005,6.032557
E01023929,1002,Welwyn Hatfield 010A,75,75.0,30860.0,E01023929,1177.0,169.0,481.0,110.0,...,31.0,12.0,11.0,4.0,18.0,40.866610,14.358539,3.313509,12.744265,0.313431
E01011976,1003,Hartlepool 005D,75,75.0,19182.0,E01011976,754.0,111.0,223.0,35.0,...,23.0,21.0,6.0,27.0,35.0,29.575597,14.721485,12.466844,11.671088,0.118284
E01011977,1004,Hartlepool 012C,75,75.0,19183.0,E01011977,1266.0,209.0,343.0,53.0,...,54.0,46.0,27.0,31.0,61.0,27.093207,16.508689,11.769352,10.110585,0.347408
E01011974,1005,Hartlepool 005B,75,37.5,19180.0,E01011974,1384.0,165.0,572.0,62.0,...,40.0,30.0,28.0,24.0,49.0,41.329480,11.921965,8.670520,9.176301,0.857310


In [46]:
final_df=pd.merge(final_df, area_df['Area (km2)'],left_index=True, right_index=True, how='left')
final_df['Area (km2)'].fillna(-1, inplace=True)
final_df.head()

,N_Households,Rurality,MSOA11CD,LAD11CD,Local Authority,Detached No central heating 2011,Detached Gas boiler 2011,Detached Resistance heating 2011,Detached Oil boiler 2011,Detached Solid fuel boiler 2011,...,Terraced Gas boiler 2011,Terraced Resistance heating 2011,Terraced Oil boiler 2011,Terraced Solid fuel boiler 2011,Flat No central heating 2011,Flat Gas boiler 2011,Flat Resistance heating 2011,Flat Oil boiler 2011,Flat Solid fuel boiler 2011,Area (km2)
LSOA11CD,,,,,,,,,,,,,,,,,,,,,
W01001694,878,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,20.0,4.0,0.0,0.0,...,396.0,10.0,0.0,1.0,33.0,220.0,72.0,0.0,0.0,0.174040
W01001695,845,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,6.0,0.0,0.0,0.0,...,152.0,2.0,0.0,0.0,17.0,290.0,286.0,8.0,0.0,0.342690
W01001696,683,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,20.0,2.0,0.0,0.0,...,338.0,12.0,0.0,0.0,3.0,174.0,25.0,0.0,0.0,0.184219
W01001697,762,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,13.0,0.0,0.0,0.0,...,466.0,5.0,0.0,0.0,7.0,124.0,51.0,0.0,0.0,0.178283
W01001698,896,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,13.0,0.0,0.0,0.0,...,225.0,5.0,0.0,1.0,33.0,384.0,152.0,1.0,0.0,0.180057


### Add road length to LSOA

In [47]:
file = 'UK_2011_LSOA_with_road.csv'

In [48]:
road_df = pd.read_csv(path_input_data+os.path.sep+file, dtype={"Sum_Shape_":float})
road_df.rename(columns={'Sum_Shape_':'Road length (m)'}, inplace=True)
road_df = pd.merge(OA_lookup_df[["OA11CD","LSOA11CD"]].drop_duplicates(), road_df[["geo_code", "Road length (m)"]], left_on="OA11CD", right_on="geo_code", how="left")
road_df = road_df.groupby(["LSOA11CD"]).sum().reset_index()
road_df['Road length (m)'].fillna(-1, inplace=True)
road_df.head()

,LSOA11CD,Road length (m)
0,W01001694,4181.887734
1,W01001695,5937.487157
2,W01001696,3269.096679
3,W01001697,3533.385626
4,W01001698,4932.216608


In [49]:
final_df = pd.merge(final_df, road_df,left_index=True, right_on="LSOA11CD", how='left')
final_df.head()

,N_Households,Rurality,MSOA11CD,LAD11CD,Local Authority,Detached No central heating 2011,Detached Gas boiler 2011,Detached Resistance heating 2011,Detached Oil boiler 2011,Detached Solid fuel boiler 2011,...,Terraced Oil boiler 2011,Terraced Solid fuel boiler 2011,Flat No central heating 2011,Flat Gas boiler 2011,Flat Resistance heating 2011,Flat Oil boiler 2011,Flat Solid fuel boiler 2011,Area (km2),LSOA11CD,Road length (m)
0,878,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,20.0,4.0,0.0,0.0,...,0.0,1.0,33.0,220.0,72.0,0.0,0.0,0.174040,W01001694,4181.887734
1,845,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,17.0,290.0,286.0,8.0,0.0,0.342690,W01001695,5937.487157
2,683,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,20.0,2.0,0.0,0.0,...,0.0,0.0,3.0,174.0,25.0,0.0,0.0,0.184219,W01001696,3269.096679
3,762,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,13.0,0.0,0.0,0.0,...,0.0,0.0,7.0,124.0,51.0,0.0,0.0,0.178283,W01001697,3533.385626
4,896,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,13.0,0.0,0.0,0.0,...,0.0,1.0,33.0,384.0,152.0,1.0,0.0,0.180057,W01001698,4932.216608


### Add number of households in 2018

In [50]:
## Add data for 2018
frames = []

year_list=['2018']

for yr in year_list:
    temp_df = pd.read_csv(path_input_data+os.path.sep+"LSOA_domestic_"+str(yr)+".csv",na_values='-')
    cols_to_keep = ["Lower Layer Super Output Area (LSOA) Code" , "Total number of domestic electricity meters"]
    temp_df = temp_df.loc[:, cols_to_keep].copy()
    temp_df.columns = ["LSOA code "+yr, 
                        "Total number of domestic electricity meters "+yr, 
                       ]
    temp_df["LSOA code "+yr]=temp_df["LSOA code "+yr].str.strip()
    temp_df.set_index("LSOA code "+yr, inplace=True)
    final_df = pd.merge(final_df, temp_df,left_on="LSOA11CD", right_on="LSOA code "+yr, how='left')
    for col in temp_df.iloc[:,:].columns:
        final_df[col].fillna(0, inplace=True)

In [51]:
final_df.rename(columns={'N_Households':'N_Households 2011'}, inplace=True) #
print("Number of households in 2011: ", '{:,}'.format(final_df['N_Households 2011'].sum()))

# the number of electricity meters is a good proxy to estimate the number of households per LSOA in 2018
final_df['N_Households 2018'] = final_df['Total number of domestic electricity meters 2018'] 
print("Number of households in 2018: ", '{:,}'.format(final_df['N_Households 2018'].sum()))

Number of households in 2011:  142,557
Number of households in 2018:  154,358


In [52]:
heating_cols = [
                'Detached Gas boiler 2011' ,
                'Detached Resistance heating 2011' ,
                'Detached Oil boiler 2011' ,
                'Detached Solid fuel boiler 2011' ,
                'Semi-detached Gas boiler 2011' ,
                'Semi-detached Resistance heating 2011' ,
                'Semi-detached Oil boiler 2011' ,
                'Semi-detached Solid fuel boiler 2011' ,
                'Terraced Gas boiler 2011' ,
                'Terraced Resistance heating 2011' ,
                'Terraced Oil boiler 2011' ,
                'Terraced Solid fuel boiler 2011' ,
                'Flat Gas boiler 2011' ,
                'Flat Resistance heating 2011' ,
                'Flat Oil boiler 2011' ,
                'Flat Solid fuel boiler 2011' ]

In [53]:
for nb_install in heating_cols:
    print(nb_install)
    final_df[nb_install[:-5]+" 2018"] = round(final_df[nb_install]/final_df["N_Households 2011"]*final_df['N_Households 2018'], 0)

Detached Gas boiler 2011
Detached Resistance heating 2011
Detached Oil boiler 2011
Detached Solid fuel boiler 2011
Semi-detached Gas boiler 2011
Semi-detached Resistance heating 2011
Semi-detached Oil boiler 2011
Semi-detached Solid fuel boiler 2011
Terraced Gas boiler 2011
Terraced Resistance heating 2011
Terraced Oil boiler 2011
Terraced Solid fuel boiler 2011
Flat Gas boiler 2011
Flat Resistance heating 2011
Flat Oil boiler 2011
Flat Solid fuel boiler 2011


In [54]:
final_df.set_index("LSOA11CD", inplace=True)
final_df.head()

,N_Households 2011,Rurality,MSOA11CD,LAD11CD,Local Authority,Detached No central heating 2011,Detached Gas boiler 2011,Detached Resistance heating 2011,Detached Oil boiler 2011,Detached Solid fuel boiler 2011,...,Semi-detached Oil boiler 2018,Semi-detached Solid fuel boiler 2018,Terraced Gas boiler 2018,Terraced Resistance heating 2018,Terraced Oil boiler 2018,Terraced Solid fuel boiler 2018,Flat Gas boiler 2018,Flat Resistance heating 2018,Flat Oil boiler 2018,Flat Solid fuel boiler 2018
LSOA11CD,,,,,,,,,,,,,,,,,,,,,
W01001694,878,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,20.0,4.0,0.0,0.0,...,0.0,0.0,446.0,11.0,0.0,1.0,248.0,81.0,0.0,0.0
W01001695,845,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,6.0,0.0,0.0,0.0,...,0.0,0.0,173.0,2.0,0.0,0.0,331.0,326.0,9.0,0.0
W01001696,683,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,20.0,2.0,0.0,0.0,...,0.0,0.0,369.0,13.0,0.0,0.0,190.0,27.0,0.0,0.0
W01001697,762,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,13.0,0.0,0.0,0.0,...,0.0,0.0,521.0,6.0,0.0,0.0,139.0,57.0,0.0,0.0
W01001698,896,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,13.0,0.0,0.0,0.0,...,0.0,0.0,263.0,6.0,0.0,1.0,448.0,177.0,1.0,0.0


## EPC data
information regarding each field is described in : https://epc.opendatacommunities.org/docs/guidance#technical_notes

In [56]:
def mergeColumn(df, target, suffixes=["_x", "_y"]):
    list_col = df.columns
    org_col = target+suffixes[0]
    repl_col = target+suffixes[1]
    boolean=False
    if org_col in list_col:
        df[target] = df[org_col].fillna(df[repl_col])
        df.drop([org_col, repl_col], axis=1, inplace=True)
        boolean=True
    return boolean  

def getEPC(path, file):
    file = 'certificates.csv'
    df = pd.read_csv(path+os.path.sep+file)
    df["POSTCODE"] = df["POSTCODE"].str.replace(' ','')
    df=pd.merge(df, OA_lookup_df.loc[:,["PCD7", "LSOA11CD", "MSOA11CD", "LAD11CD"]],left_on="POSTCODE", right_on="PCD7", how='left')

    df.drop(["PCD7"], axis=1, inplace=True)
    df.replace('Bungalow', 'House', regex=True, inplace=True)
    df.replace('Maisonette', 'Flat', regex=True, inplace=True)
    df["Heat consumption (kWh)"]=0
    df["Space constrained ratio"]=0
    df["Heat_rating_changes [%]"]=0
    df["Heat_efficiency_changes [%]"]=0
    print(df.shape)
    return df

    
def setFlag(df):
    #Remove rows without a valid LSOA, MSOA or LA
    df["FLAG LSOA"] = df["LSOA11CD"].astype(str).str.len()==9
    df.drop(df[df["FLAG LSOA"]==False].index, inplace=True)

    df["FLAG MSOA"] = df["MSOA11CD"].astype(str).str.len()==9
    df.drop(df[df["FLAG MSOA"]==False].index, inplace=True)

    df["FLAG LA"] = df["LAD11CD"].astype(str).str.len()==9
    df.drop(df[df["FLAG LA"]==False].index, inplace=True)
    
    return df

def costTokWh(df, df_fuels):
    print("list of fuels: ", df["MAIN_FUEL"].unique()) 
    if potential:
        cost_column = "HEATING_COST_POTENTIAL"
    else:
        cost_column = "HEATING_COST_CURRENT" 
    #USING SAP price from january 2016: https://www.bre.co.uk/filelibrary/SAP/2012/RdSAP-fuel-prices-from-January-2018.xlsx
    price_of_elec = 15.32/100 #df_fuels.loc["Electricity", "Cost [GBP/kWh]"]
    price_of_gas = 4.32/100 #df_fuels.loc["Ngas", "Cost [GBP/kWh]"] 
    price_of_oil = 5.06/100 #df_fuels.loc["Oil", "Cost [GBP/kWh]"] 
    price_of_solid_fuel = 5.67/100 #df_fuels.loc["Biomass", "Cost [GBP/kWh]"] 

    #Efficiency from "combined"  individual_tech database
    eff_elec = 1
    eff_gas = 0.84
    eff_oil = 0.84
    eff_solid_fuel = 0.82
    eff_hp = 3.4
    
    df["Heat system"] = None
    df["MAINHEAT_DESCRIPTION"].fillna("No value", inplace=True)
    df["Heat consumption (kWh)"].dropna(inplace=True)
    
    gas_list = ["mains gas (not community)",
                #'mains gas - this is for backwards compatibility only and should not be used'
                ]
    
    df.loc[df["MAIN_FUEL"].isin(gas_list), "Heat consumption (kWh)"] = df.loc[df["MAIN_FUEL"].isin(gas_list), cost_column]/price_of_gas*eff_gas
    df.loc[df["MAIN_FUEL"].isin(gas_list), "Heat system"] = "Gas boiler"

    electricity_list = ["electricity (not community)", 'electricity - this is for backwards compatibility only and should not be used',
                       'Electricity: electricity, unspecified tariff']
    
    df.loc[df["MAIN_FUEL"].isin(electricity_list), "Heat system"]='Resistance heating'
    df.loc[df["MAINHEAT_DESCRIPTION"].str.contains("heat pump"),"Heat system"]="Heat pump"
    
    df.loc[df["Heat system"]=='Resistance heating', "Heat consumption (kWh)"] = df.loc[df["Heat system"]=='Resistance heating', cost_column]/price_of_elec*eff_elec
    df.loc[df["Heat system"]=="Heat pump", "Heat consumption (kWh)"] = df.loc[df["Heat system"]=="Heat pump", cost_column]/price_of_elec*eff_hp

    oil_list = ['appliances able to use mineral oil or liquid biofuel','oil (not community)', 'oil - this is for backwards compatibility only and should not be used']
    
    df.loc[df["MAIN_FUEL"].isin(oil_list), "Heat system"] = "Oil boiler"
    df.loc[df["Heat system"]=="Oil boiler", "Heat consumption (kWh)"] = df.loc[df["Heat system"]=="Oil boiler", cost_column]/price_of_oil*eff_oil

    solid_fuels = ['anthracite', 'house coal (not community)','wood logs', 'smokeless coal', 'house coal - this is for backwards compatibility only and should not be used',
                  'bulk wood pellets', 'wood chips']

    df.loc[df["MAIN_FUEL"].isin(solid_fuels), "Heat system"]="Solid fuel boiler"
    df.loc[df["Heat system"]=="Solid fuel boiler", "Heat consumption (kWh)"] = df.loc[df["Heat system"]=="Solid fuel boiler", cost_column]/price_of_solid_fuel*eff_solid_fuel
    return df

def threshold(df):
    # Remove inaccurate value from the dataset. e.g. everything under 25 kWh/m2
    min_kWh_m2 = 25 # minimum intensity Passive House (Passivhaus) standard is at 15 kWh/m2
    max_kWh_m2 = 400 
    max_m2 = 1000
    min_number_room = 1
    
    df["Heat per floor area (kWh/m2)"]=df["Heat consumption (kWh)"]/df["TOTAL_FLOOR_AREA"]
    print(df.loc[(df["Heat per floor area (kWh/m2)"]<min_kWh_m2),:].shape)
    df.fillna(0, inplace=True)
    to_drop = df.loc[(df["Heat per floor area (kWh/m2)"]<min_kWh_m2),:].index
    df.drop(to_drop, inplace=True)
    print(df.loc[(df["Heat per floor area (kWh/m2)"]>max_kWh_m2),:].shape)
    to_drop = df.loc[(df["Heat per floor area (kWh/m2)"]>max_kWh_m2),:].index
    df.drop(to_drop, inplace=True)
       
    to_drop = df.loc[(df["HEATING_COST_POTENTIAL"]>df["HEATING_COST_CURRENT"]),:].index
    df.drop(to_drop, inplace=True)
    
    # Used to assess the sapce availability to change heating system 
    # Source: Element Energy, “Analysis on abating direct emissions from ‘hard-to-decarbonise’ homes, with a view to informing the UK’s long term targets,” 2019.
    df["Space constrained ratio"] = df["TOTAL_FLOOR_AREA"]/df["NUMBER_HEATED_ROOMS"]
    df.loc[df["Space constrained ratio"]<=16, "Space constrained ratio"]=1
    df.loc[df["Space constrained ratio"]>16, "Space constrained ratio"]=0
    
    # Change in EPC and impact on heat
    to_drop = df.loc[df["POTENTIAL_ENERGY_EFFICIENCY"]<df["CURRENT_ENERGY_EFFICIENCY"], :].index
    df.drop(to_drop, inplace=True)
    df["Heat_rating_changes [%]"] = (df["HEATING_COST_POTENTIAL"] - df["HEATING_COST_CURRENT"])/df["HEATING_COST_CURRENT"]
    df["Heat_efficiency_changes [%]"] = (df["HEATING_COST_POTENTIAL"] - df["HEATING_COST_CURRENT"])/df["HEATING_COST_CURRENT"]/(df["POTENTIAL_ENERGY_EFFICIENCY"] - df["CURRENT_ENERGY_EFFICIENCY"])

    df["Heat_rating_changes [%]"].replace([np.inf, -np.inf], 0, inplace=True)
    df["Heat_efficiency_changes [%]"].replace([np.inf, -np.inf], 0, inplace=True)
    #Remove houses with less than 1 heated room
    to_drop = df.loc[(df["NUMBER_HEATED_ROOMS"]<min_number_room),:].index
    df.drop(to_drop, inplace=True)

    return df


def dwellingType(df):
    
    df["Dwelling type"]=None
    df.loc[df["PROPERTY_TYPE"]=="Flat", "Dwelling type"]="Flat"
    df.loc[df["PROPERTY_TYPE"]=="House", "Dwelling type"]=df.loc[df["PROPERTY_TYPE"]=="House", "BUILT_FORM"]
    list_dwelling_type = list(df["Dwelling type"].unique())
    list_dwelling_type = list(filter(None.__ne__, list_dwelling_type)) # Remove None from the list
    # temp_df.replace({dwel:"Terrace" for dwel in list_dwelling_type if "Terrace" in dwel}, regex=True, inplace=True)
   
    to_replace = {dwel:"Terraced" for dwel in list_dwelling_type if "Terrace" in dwel}
    df.replace(to_replace, inplace=True)
    df.replace("Semi-Detached", "Semi-detached", inplace=True)
    list_dwelling_type = list(df["Dwelling type"].unique())
    print(list_dwelling_type)
    
    to_drop = df.loc[(df["Dwelling type"]=='NO DATA!'),:].index
    df.drop(to_drop, inplace=True)
    
    df["Key"] = df["Dwelling type"]+" "+df["Heat system"]
    df["Key_EPC"] = df["CURRENT_ENERGY_RATING"]+" to "+df["POTENTIAL_ENERGY_RATING"]
    df.dropna(subset=['Key'], inplace=True)
    print("Shape:", df.shape)
    
    
    return df

def skeleton(LSOA_lookup):
    LSOA_lookup = LSOA_lookup.iloc[:,0:4] # use only the four first columns
    index = list(map(list, zip(*LSOA_lookup.values)))
    index = pd.MultiIndex.from_arrays(index, names=LSOA_lookup.columns)

    heating_type = ["Gas boiler", 'Resistance heating', "Oil boiler", "Solid fuel boiler", "Heat pump"]
    list_col = ["Mean "+x+" "+y for x in dwelling_type for y in heating_type]
#     list_col.extend(["Floor area "+x+" "+y for x in dwelling_type for y in heating_type])
#     list_col.extend(["Number of rooms "+x+" "+y for x in dwelling_type for y in heating_type])
#     list_col.extend(["Space constrained "+x+" "+y for x in dwelling_type for y in heating_type])
#     EPC_ratings = ["A", "B", "C", "D", "E", "F", "G"]
#     EPC_rating_cols = []
#     EPC_eff_cols = []
#     for ii, val in enumerate(EPC_ratings):
#         for val2 in EPC_ratings[ii:]:
#             EPC_rating_cols.append("EPC rating "+val2+" to "+val)
#             EPC_eff_cols.append("EPC efficiency "+val2+" to "+val)
#     list_col.extend(EPC_rating_cols)
#     list_col.extend(EPC_eff_cols)
    LSOA_LA_df = pd.DataFrame(index=index, columns=list_col)
    LSOA_LA_df.reset_index(3,drop=True, inplace=True)
    LSOA_LA_df = LSOA_LA_df.reorder_levels(["LAD11CD",  'MSOA11CD','LSOA11CD'])
    return LSOA_LA_df


def populatingSkeleton(df, skeleton, target):
    list_col = list(skeleton.columns)
    key = "Key"
    if target=="Heat consumption (kWh)":
        list_col = [x for x in list_col if "Mean" in x]
        prefix = "Mean"
    elif target=="NUMBER_HEATED_ROOMS":
        list_col = [x for x in list_col if "rooms" in x]
        prefix = "Number of rooms"
    elif target=="TOTAL_FLOOR_AREA":
        list_col = [x for x in list_col if "Floor" in x]
        prefix = "Floor area"
    elif target=="Space constrained ratio":
        list_col = [x for x in list_col if "Space constrained" in x]
        prefix = "Space constrained"
    elif target=="Heat_rating_changes [%]":
        
        list_col = [x for x in list_col if "EPC rating" in x]
        prefix = "EPC rating"
        key= "Key_EPC"
    elif target=="Heat_efficiency_changes [%]":
        list_col = [x for x in list_col if "EPC efficiency" in x]
        prefix = "EPC efficiency"
        key= "Key_EPC"
        
    print(list_col, prefix)
    df[key] = [prefix+" "+x for x in df[key].values]
    print(df[key].unique())
    
    df = df.loc[df[key].isin(list_col), :].copy()
    # Add the value available at LSOA level
    LSOA_df_count = df.groupby(["LAD11CD", "MSOA11CD", "LSOA11CD", key]).count()[target]
    LSOA_df_count[LSOA_df_count<4]=np.nan #Remove the archetypes with too few properties
    LSOA_df = df.groupby(["LAD11CD", "MSOA11CD", "LSOA11CD", key]).sum()[target]
    LSOA_df = LSOA_df/LSOA_df_count
    LSOA_df = LSOA_df.to_frame()
    LSOA_df = LSOA_df.unstack(3)
    LSOA_df.columns = LSOA_df.columns.droplevel(0)
    
    populated_skeleton = pd.merge(skeleton, LSOA_df, left_index=True, right_index=True, how='left')
    for col in list_col:
        mergeColumn(populated_skeleton, col)
    populated_skeleton.reset_index(level=2, inplace=True)
    
    # Add the value available at MSOA level
    MSOA_df_count = df.groupby(["LAD11CD", "MSOA11CD", key]).count()[target]
    MSOA_df_count[MSOA_df_count<4]=np.nan #Remove the archetypes with too few properties
    MSOA_df = df.groupby(["LAD11CD", "MSOA11CD", key]).sum()[target]
    MSOA_df = MSOA_df/MSOA_df_count
    MSOA_df = MSOA_df.to_frame()
    MSOA_df = MSOA_df.unstack(2)
    MSOA_df.columns = MSOA_df.columns.droplevel(0)

    populated_skeleton = pd.merge(populated_skeleton, MSOA_df, left_index=True, right_index=True, how='left')
    for col in list_col:
        mergeColumn(populated_skeleton, col)
        
    populated_skeleton.reset_index(level=1, inplace=True)
    
    # Add the value available at LA level
    LA_df_count = df.groupby(["LAD11CD", key]).count()[target]
    LA_df_count[LA_df_count<4]=np.nan #Remove the archetypes with too few properties
    LA_df = df.groupby(["LAD11CD", key]).sum()[target]
    LA_df = LA_df/LA_df_count
    LA_df = LA_df.to_frame()
    LA_df = LA_df.unstack(1)
    LA_df.columns = LA_df.columns.droplevel(0)
    LA_df.fillna(LA_df.mean() ,inplace=True)
    
    populated_skeleton = pd.merge(populated_skeleton, LA_df, left_index=True, right_index=True, how='left')
    for col in list_col:
        mergeColumn(populated_skeleton, col)
        
    populated_skeleton.reset_index(inplace=True)
    populated_skeleton.set_index(["LAD11CD", "MSOA11CD", "LSOA11CD"], inplace=True, drop=True)
    
    populated_skeleton["Mean Flat Oil boiler"].fillna(populated_skeleton["Mean Flat Solid fuel boiler"], inplace=True)
    
    return populated_skeleton

def epcToLSOA(path, fn, LSOA_lookup):
    epc_df = getEPC(path, fn)
    epc_df = setFlag(epc_df)
    epc_df = costTokWh(epc_df, df_fuels)
    epc_df = threshold(epc_df)
    epc_df = dwellingType(epc_df)
    skeleton_df = skeleton(LSOA_lookup)
    df = populatingSkeleton(epc_df.copy(), skeleton_df, "Heat consumption (kWh)")
#     df = populatingSkeleton(epc_df.copy(), df, "NUMBER_HEATED_ROOMS")
#     df = populatingSkeleton(epc_df.copy(), df, "TOTAL_FLOOR_AREA")
#     df = populatingSkeleton(epc_df.copy(), df, "Space constrained ratio")
#     df = populatingSkeleton(epc_df.copy(), df, "Heat_rating_changes [%]")
#     df = populatingSkeleton(epc_df.copy(), df, "Heat_efficiency_changes [%]")
    df.reset_index(inplace=True)
    df.set_index("LSOA11CD", inplace=True)
    df.drop(["MSOA11CD", "LAD11CD"], axis=1, inplace=True)

    return df

In [59]:
fuel_cost_file = "fuel_costs.csv"

#import the dataset related to the fuel costs
df_fuels = pd.read_csv(path_input_data+os.path.sep+fuel_cost_file, index_col=0)
df_fuels.head()

,Cost [GBP/kWh],Emissions [kg-CO2e/kWh]
Fuel,,
Ngas,0.043600,0.20437
Electricity,0.178886,0.28307
Biomass,0.050000,0.01506
Oil,0.036130,0.25963


In [61]:
# uniformisation of the string values

list_LA = OA_lookup_df["Local Authority"].unique()
list_LA = [x.replace(' ', '') for x in list_LA]
list_LA = [x.replace('-', '') for x in list_LA]
list_LA = [x.replace("'", '') for x in list_LA]
list_LA = [x.replace(".", '') for x in list_LA]

LSOA_lookup_df["Local Authority nospace"] = LSOA_lookup_df["Local Authority"].str.replace('-','')
LSOA_lookup_df["Local Authority nospace"] = LSOA_lookup_df["Local Authority nospace"].str.replace(".",'')
LSOA_lookup_df["Local Authority nospace"] = LSOA_lookup_df["Local Authority nospace"].str.replace("'",'')
LSOA_lookup_df["Local Authority nospace"] = LSOA_lookup_df["Local Authority nospace"].str.replace(' ','')

C:\Users\sceac10\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  # Remove the CWD from sys.path while we load stuff.


In [62]:
# output_df.set_index("LSOA11CD", inplace=True)
for fn in os.listdir(path_EPCs):
    LA_name = fn.split('-')[2:]
    LA_name = ''.join(LA_name)
    print(LA_name)
    if LA_name in list_LA:
        LSOA_lookup = LSOA_lookup_df.loc[LSOA_lookup_df["Local Authority nospace"]==LA_name, :]
        if LSOA_lookup.shape[0]>0:
            temp_df = epcToLSOA(path_EPCs+os.path.sep+fn+os.path.sep, file, LSOA_lookup)
            final_df = pd.merge(final_df, temp_df, left_index=True, right_index=True, how='left')
            for col in temp_df.columns:
                mergeColumn(final_df, col)
        else: print("ERROR WITH NAME !!!!!!!!!!!!!!!!!!!", LA_name)
    else:
        print("Out", LA_name, fn)

Cardiff
(101621, 92)
list of fuels:  ['mains gas - this is for backwards compatibility only and should not be used'
 'electricity (not community)'
 'electricity - this is for backwards compatibility only and should not be used'
 'mains gas (not community)' 'NO DATA!'
 'Electricity: electricity, unspecified tariff'
 'To be used only when there is no heating/hot-water system'
 'oil (not community)' 'INVALID!' 'smokeless coal'
 'dual fuel - mineral + wood' 'mains gas (community)'
 'oil - this is for backwards compatibility only and should not be used'
 'LPG (not community)'
 'house coal - this is for backwards compatibility only and should not be used'
 'wood logs'
 'LPG - this is for backwards compatibility only and should not be used'
 'house coal (not community)' 'bulk wood pellets' 'wood chips'
 'electricity (community)' 'wood pellets in bags for secondary heating'
 'oil (community)' 'bottled LPG' 'anthracite']
(41257, 97)
(270, 97)
['Semi-detached', 'Flat', 'Terraced', 'Detached', 'N

In [63]:
cols = ['Flat Heat pump', 'Detached Gas boiler', 'Detached Resistance heating', 'Detached Oil boiler', 'Detached Solid fuel boiler',
 'Detached Heat pump', 'Semi-detached Gas boiler', 'Semi-detached Resistance heating', 'Semi-detached Oil boiler',
 'Semi-detached Solid fuel boiler', 'Semi-detached Heat pump', 'Terraced Gas boiler', 'Terraced Resistance heating',
 'Terraced Oil boiler', 'Terraced Solid fuel boiler', 'Terraced Heat pump', 'Flat Gas boiler', 'Flat Resistance heating', 'Flat Oil boiler',
 'Flat Solid fuel boiler']

for p in ["Mean"]:#,"Number of rooms", "Floor area", "Space constrained"]:
    dict_avg_demand = {}
    temp_cols = [p+" "+x for x in cols]
    for col in temp_cols:
        dict_avg_demand[col]=final_df[col].mean()
    print(dict_avg_demand)

{'Mean Flat Heat pump': 3533.18001024864, 'Mean Detached Gas boiler': 12535.349394998682, 'Mean Detached Resistance heating': 5528.508280152917, 'Mean Detached Oil boiler': 23913.649677757578, 'Mean Detached Solid fuel boiler': 11394.554418093987, 'Mean Detached Heat pump': 30681.388936336283, 'Mean Semi-detached Gas boiler': 10871.450087693223, 'Mean Semi-detached Resistance heating': 4151.004054508962, 'Mean Semi-detached Oil boiler': 14272.935414849197, 'Mean Semi-detached Solid fuel boiler': 6041.139646120365, 'Mean Semi-detached Heat pump': 12792.167101827627, 'Mean Terraced Gas boiler': 9527.160679396851, 'Mean Terraced Resistance heating': 3721.206058775337, 'Mean Terraced Oil boiler': 10399.645376971705, 'Mean Terraced Solid fuel boiler': 7955.074969644021, 'Mean Terraced Heat pump': nan, 'Mean Flat Gas boiler': 6318.843944489338, 'Mean Flat Resistance heating': 2147.703098123723, 'Mean Flat Oil boiler': 5876.679841897223, 'Mean Flat Solid fuel boiler': 3644.4444444444575}


In [64]:
if potential: #average value in England and Wales after considering EE measures (obtained from a previous run)
    avg_heat_demand ={'Mean Flat Heat pump': 5501.065403187484, 
                       'Mean Detached Gas boiler': 14632.191288588996, 
                       'Mean Detached Resistance heating': 6204.286121513095, 
                       'Mean Detached Oil boiler': 21098.605684646347, 
                       'Mean Detached Solid fuel boiler': 14041.56080301572, 
                       'Mean Detached Heat pump': 28783.898824454503, 
                       'Mean Semi-detached Gas boiler': 11218.187752226073, 
                       'Mean Semi-detached Resistance heating': 4295.142660970292, 
                       'Mean Semi-detached Oil boiler': 13677.813694605993, 
                       'Mean Semi-detached Solid fuel boiler': 8637.525085372072, 
                       'Mean Semi-detached Heat pump': 14392.297099877122, 
                       'Mean Terraced Gas boiler': 9824.556795534983, 
                       'Mean Terraced Resistance heating': 3748.5168135325443, 
                       'Mean Terraced Oil boiler': 11902.605926956383, 
                       'Mean Terraced Solid fuel boiler': 7891.128193748849,
                       'Mean Terraced Heat pump': 11142.367353892165, 
                       'Mean Flat Gas boiler': 6589.520748462197, 
                       'Mean Flat Resistance heating': 2331.4149814824323, 
                       'Mean Flat Oil boiler': 6853.035108582689, 
                       'Mean Flat Solid fuel boiler': 4956.443883595216}
    
else: #average value in England and Wales before considering EE measures (obtained from a previous run)
    avg_heat_demand = {'Mean Flat Heat pump': 6719.579873742368, 
                       'Mean Detached Gas boiler': 21530.680745512054, 
                       'Mean Detached Resistance heating': 10431.509354267522, 
                       'Mean Detached Oil boiler': 32602.880527916703, 
                       'Mean Detached Solid fuel boiler': 21191.994977242324, 
                       'Mean Detached Heat pump': 35559.90356372218, 
                       'Mean Semi-detached Gas boiler': 16313.371301143585,
                       'Mean Semi-detached Resistance heating': 7109.6953266522805,
                       'Mean Semi-detached Oil boiler': 21169.557721387726, 
                       'Mean Semi-detached Solid fuel boiler': 12940.565949958354, 
                       'Mean Semi-detached Heat pump': 18589.901697392448, 
                       'Mean Terraced Gas boiler': 13949.472235960391, 
                       'Mean Terraced Resistance heating': 5928.109571397592, 
                       'Mean Terraced Oil boiler': 17516.285501274226, 
                       'Mean Terraced Solid fuel boiler': 11488.08155763224,
                       'Mean Terraced Heat pump': 14359.707672412354, 
                       'Mean Flat Gas boiler': 8970.309878187718, 
                       'Mean Flat Resistance heating': 3394.009767843848, 
                       'Mean Flat Oil boiler': 9822.074874718057, 
                       'Mean Flat Solid fuel boiler': 6797.2720595054625}
    

for key, value in avg_heat_demand.items():
    final_df[key].fillna(value, inplace=True)

In [65]:
# Remove dwellings with heat pumps
heatpump_cols = [c for c in final_df.columns if "Heat pump" in c]
final_df.drop(heatpump_cols, axis=1, inplace=True)

### Formatting of the name of the columns

In [66]:
current_categories = ['Detached Gas boiler' ,
                        'Detached Resistance heating' ,
                        'Detached Oil boiler' ,
                        'Detached Biomass boiler' ,
                        'Semi-detached Gas boiler' ,
                        'Semi-detached Resistance heating' ,
                        'Semi-detached Oil boiler' ,
                        'Semi-detached Biomass boiler' ,
                        'Terraced Gas boiler' ,
                        'Terraced Resistance heating' ,
                        'Terraced Oil boiler' ,
                        'Terraced Biomass boiler' ,
                        'Flat Gas boiler' ,
                        'Flat Resistance heating' ,
                        'Flat Oil boiler' ,
                        'Flat Biomass boiler' ]

In [67]:
final_df.columns = [c.replace("Solid fuel", "Biomass") for c in final_df.columns]

rename_dict = {}
for c in current_categories:
    if potential:
        rename_dict["Mean "+c] = "Average heat demand after energy efficiency measures for "+c.lower()+" (kWh)"
    else:
        rename_dict["Mean "+c] = "Average heat demand before energy efficiency measures for "+c.lower()+" (kWh)"

final_df.rename(columns=rename_dict, inplace=True)


In [68]:
final_df

,N_Households 2011,Rurality,MSOA11CD,LAD11CD,Local Authority,Detached No central heating 2011,Detached Gas boiler 2011,Detached Resistance heating 2011,Detached Oil boiler 2011,Detached Biomass boiler 2011,...,Average heat demand after energy efficiency measures for semi-detached oil boiler (kWh),Average heat demand after energy efficiency measures for semi-detached biomass boiler (kWh),Average heat demand after energy efficiency measures for terraced gas boiler (kWh),Average heat demand after energy efficiency measures for terraced resistance heating (kWh),Average heat demand after energy efficiency measures for terraced oil boiler (kWh),Average heat demand after energy efficiency measures for terraced biomass boiler (kWh),Average heat demand after energy efficiency measures for flat gas boiler (kWh),Average heat demand after energy efficiency measures for flat resistance heating (kWh),Average heat demand after energy efficiency measures for flat oil boiler (kWh),Average heat demand after energy efficiency measures for flat biomass boiler (kWh)
LSOA11CD,,,,,,,,,,,,,,,,,,,,,
W01001694,878,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,20.0,4.0,0.0,0.0,...,14320.097294,6125.396825,10074.828179,2286.460276,10420.316206,8205.784832,6062.345679,1810.116574,5876.679842,3644.444444
W01001695,845,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,6.0,0.0,0.0,0.0,...,14320.097294,6125.396825,9628.949653,2961.995938,10420.316206,8205.784832,5644.085249,1635.577171,5876.679842,3644.444444
W01001696,683,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,20.0,2.0,0.0,0.0,...,14320.097294,6125.396825,8764.442432,3330.287206,10420.316206,8205.784832,5603.120713,2012.930499,5876.679842,3644.444444
W01001697,762,Rurality_Urban >10K,W02000402,W06000015,Cardiff,0.0,13.0,0.0,0.0,0.0,...,14320.097294,6125.396825,9862.180256,2961.995938,10420.316206,8205.784832,5810.457516,2029.339013,5876.679842,3644.444444
W01001698,896,Rurality_Urban >10K,W02000402,W06000015,Cardiff,1.0,13.0,0.0,0.0,0.0,...,14320.097294,6125.396825,12159.033816,2961.995938,10420.316206,8205.784832,6152.280265,1964.989319,5876.679842,3644.444444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W01001950,609,"Rurality_Village, Town and Fringe",W02000423,W06000015,Cardiff,0.0,2.0,0.0,0.0,0.0,...,14320.097294,6125.396825,9504.310345,3034.522773,10420.316206,8205.784832,7909.027778,3032.800261,5876.679842,3644.444444
W01001951,548,Rurality_Urban >10K,W02000423,W06000015,Cardiff,0.0,4.0,0.0,0.0,0.0,...,14320.097294,6125.396825,7957.924837,2864.848152,10420.316206,8205.784832,6154.202542,2125.987589,5876.679842,3644.444444
W01001952,621,Rurality_Urban >10K,W02000423,W06000015,Cardiff,0.0,7.0,0.0,0.0,0.0,...,14320.097294,6125.396825,10346.604938,3034.522773,10420.316206,8205.784832,8626.851852,1861.245804,5876.679842,3644.444444


In [69]:
final_df.to_csv(saving_path+os.path.sep+output_file_name+".csv")